In [46]:
import pandas as pd
import numpy  as np

import data file

In [47]:
df = pd.read_csv('data (1).csv')
df.describe()

,Label
count,333.000000
mean,0.582583
std,0.493875
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


split data

In [48]:
print("Non-cheat: ", len(df[df['Label']==0]))
print("Cheat: ", len(df[df['Label']==1]))

# diff = abs(len(df[df['Label']==0]) - len(df[df['Label']==1]))

grouped = df.groupby(df.Label)
df1 = grouped.get_group(1)
df2 = grouped.get_group(0)

df2_1 = df.iloc[:139,:]
df2_2 = df.iloc[139:194,:]

df1_train = df1.iloc[:111,:]
df2_train = df2_1.iloc[:111,:]

df1_test = df1.iloc[111:139,:]
df2_test = df2_1.iloc[111:139,:]

train = pd.concat([df1_train, df2_train])
test  = pd.concat([df1_test, df2_test, df2_2])

print((len(train), len(test)))

Non-cheat:  139
Cheat:  194
(222, 111)


parse data from string to numpy array

In [49]:
def parseData(series):
    final_series = []
    for ser in series:
        temp = []
        for x in "[],":
            ser = ser.replace(x, "")
        new_ser  = np.fromstring(ser, dtype=float, sep=" ")
        for i in range(0, len(new_ser), 3):
            chunk = [new_ser[i], new_ser[i + 1], new_ser[i + 2]]
            temp.append(chunk)
        final_series.append(temp)
    return np.array(final_series, dtype=object)

get the longest sequence of vector in dataset

In [50]:
max_length = 0
for lgaze in parseData(df['Left Gaze']):
    if len(lgaze) > max_length:
        max_length = len(lgaze)
        
max_length

1928

pad the dataset to the longest sequence size

In [51]:
import math

def padData(series, length):
    for idx, ser in enumerate(series):
        times = math.floor(length / len(ser))
        add = length % len(ser)
        
        temp = ser[::-1]
        for _ in range(1, times):
            series[idx] = np.append(series[idx], temp, axis=0)
            temp = temp[::-1]
        if add != 0:
            series[idx] = np.append(series[idx], temp[0:add], axis=0)
    return series

Convolution test

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, AveragePooling1D, LSTM
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf

def init_model():
    model = Sequential()
    model.add(Conv1D(filters=12, kernel_size=3, padding="same", activation='relu', input_shape=(max_length, 12)))
    model.add(AveragePooling1D(pool_size=2))
    model.add(Conv1D(filters=12, kernel_size=3, padding="same", activation='relu', input_shape=(max_length, 12)))
    model.add(AveragePooling1D(pool_size=2))
    
    model.add(LSTM(12))
    
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    
    model.add(Flatten())
    model.add(Dense(2, activation = 'softmax')) # We have only 2 classes: Non-cheat & Cheat
    
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [53]:
right_gaze_train = train["Right Gaze"]
left_gaze_train  = train["Left Gaze"]
right_head_train = train["Right HeadPose"]
left_head_train  = train["Left HeadPose"]

label_train = train['Label']

right_gaze_train, left_gaze_train, right_head_train, left_head_train = parseData(right_gaze_train), parseData(left_gaze_train), parseData(right_head_train), parseData(left_head_train)

initialize new data frame

In [54]:
left_head_train  = padData(left_head_train,  max_length)
right_head_train = padData(right_head_train, max_length)
left_gaze_train  = padData(left_gaze_train,  max_length)
right_gaze_train = padData(right_gaze_train, max_length)

In [55]:
label_train = np.array(label_train)

In [56]:
y_train = []
for i in range(len(label_train)):
    y_train.append(
        tf.convert_to_tensor(
            np.reshape(tf.keras.utils.to_categorical(label_train[i], num_classes=2), (1, 2))                 
                            )
             )
    
y_train = tf.convert_to_tensor(np.vstack(y_train), dtype=tf.float32)

In [57]:
x_train = []

for i in range(len(left_gaze_train)):
    x_train.append(tf.convert_to_tensor([
                                            np.hstack(
                                                        (left_gaze_train[i], right_gaze_train[i], left_head_train[i], right_head_train[i])
                                                     )
                                            ], dtype=tf.float32))



x_train = tf.convert_to_tensor(np.vstack(x_train), dtype=tf.float32)

print(x_train)

print(y_train)

tf.Tensor(
[[[  8.705  28.278  29.588 ...   1.558   2.732   3.145]
  [ -0.833  30.56  -30.571 ... -10.108   5.971 -11.741]
  [ -3.198  34.952 -35.098 ... -10.394   8.097 -13.176]
  ...
  [  5.763  14.284  15.403 ...  -1.548  -4.796  -5.04 ]
  [  1.595  16.009  16.089 ...  -2.135  -2.671  -3.42 ]
  [  2.936  15.549  15.824 ...  -1.135  -3.142  -3.34 ]]

 [[ 22.966  10.843  25.397 ...  30.359  -4.118  30.637]
  [  5.671  15.266  16.285 ...  18.896  -1.472  18.953]
  [  3.35   10.455  10.978 ...  -1.709   5.526  -5.784]
  ...
  [ -2.163  16.068 -16.213 ...  -6.742  17.116 -18.396]
  [ -2.357  15.856 -16.03  ...  -7.187  17.164 -18.608]
  [ -7.195  17.968 -19.355 ... -11.295  20.196 -23.14 ]]

 [[ 27.556  25.147  37.306 ...  17.437  34.229  38.415]
  [ 20.462  44.667  49.131 ...   2.079  50.654  50.697]
  [ 15.712  58.805  60.868 ...  -3.587  62.252 -62.355]
  ...
  [ 13.106  43.365  45.302 ...  14.919  11.971  19.128]
  [  9.422  17.793  20.134 ...   4.489  18.852  19.379]
  [ 14.79   14.

In [58]:
model = init_model()
callbacks = [EarlyStopping('val_loss', patience=3)]
# model.fit(final_df, label, epochs=10, batch_size=128, validation_split=0.2, callbacks=callbacks)
model.fit(x_train, y_train, epochs=100, batch_size=128, callbacks=callbacks)

Epoch 1/100


e:\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


2/2 [==============================] - 4s 287ms/step - loss: 0.6990 - accuracy: 0.5541
Epoch 2/100
2/2 [==============================] - 1s 271ms/step - loss: 0.6423 - accuracy: 0.6306
Epoch 3/100
2/2 [==============================] - 1s 258ms/step - loss: 0.6048 - accuracy: 0.7117
Epoch 4/100
2/2 [==============================] - 1s 271ms/step - loss: 0.5754 - accuracy: 0.7387
Epoch 5/100
2/2 [==============================] - 1s 261ms/step - loss: 0.5538 - accuracy: 0.7523
Epoch 6/100
2/2 [==============================] - 1s 258ms/step - loss: 0.5336 - accuracy: 0.7703
Epoch 7/100
2/2 [==============================] - 1s 275ms/step - loss: 0.5233 - accuracy: 0.7883
Epoch 8/100
2/2 [==============================] - 1s 256ms/step - loss: 0.5125 - accuracy: 0.7928
Epoch 9/100
2/2 [==============================] - 1s 259ms/step - loss: 0.4998 - accuracy: 0.7973
Epoch 10/100
2/2 [==============================] - 1s 251ms/step - loss: 0.4932 - accuracy: 0.7883
Epoch 11/100
2/2 [===

In [59]:
for layer in model.layers:
    print(layer.output_shape)

(None, 1928, 12)
(None, 964, 12)
(None, 964, 12)
(None, 482, 12)
(None, 12)
(None, 12)
(None, 2)


In [60]:
right_gaze_test = test["Right Gaze"]
left_gaze_test  = test["Left Gaze"]
right_head_test = test["Right HeadPose"]
left_head_test  = test["Left HeadPose"]

label_test = test['Label']

right_gaze_test, left_gaze_test, right_head_test, left_head_test = parseData(right_gaze_test), parseData(left_gaze_test), parseData(right_head_test), parseData(left_head_test)

In [61]:
left_head_test  = padData(left_head_test,  max_length)
right_head_test = padData(right_head_test, max_length)
left_gaze_test  = padData(left_gaze_test,  max_length)
right_gaze_test = padData(right_gaze_test, max_length)

In [63]:
x_test = []

for i in range(len(left_gaze_test)):
    x_test.append(tf.convert_to_tensor([
                                            np.hstack(
                                                        (left_gaze_test[i], right_gaze_test[i], left_head_test[i], right_head_test[i])
                                                     )
                                            ], dtype=tf.float32))



x_test = tf.convert_to_tensor(np.vstack(x_test), dtype=tf.float32)

print(x_test)

tf.Tensor(
[[[ 51.537  30.285  59.777 ...  53.407   8.105  54.019]
  [ 40.51   37.984  55.533 ...  48.782   6.237  49.179]
  [ 26.477  32.743  42.108 ...  41.713  14.326  44.104]
  ...
  [ -8.213  84.847 -85.244 ...  -5.171  60.839 -61.058]
  [ -9.181  83.206 -83.711 ...  -7.434  60.092 -60.55 ]
  [ -6.834  80.71  -80.999 ...  -4.513  58.137 -58.312]]

 [[ 29.547  18.415  34.815 ...  44.61    5.529  44.951]
  [ 28.973  18.207  34.219 ...  43.922   4.782  44.182]
  [ 43.92   15.941  46.724 ...  48.461   1.124  48.474]
  ...
  [ 22.409  13.187  26.001 ...  31.08  -14.119  34.137]
  [ 16.131  11.608  19.874 ...  28.504 -12.468  31.112]
  [ 18.155  14.661  23.336 ...  31.591 -12.036  33.806]]

 [[-16.954  73.426 -75.358 ... -26.907  60.171 -65.913]
  [-22.978  82.974 -86.096 ... -32.98   67.892 -75.478]
  [-39.21   73.834 -83.599 ... -38.416  60.714 -71.847]
  ...
  [-23.261  57.586 -62.107 ... -17.402  90.106 -91.771]
  [-12.14   59.276 -60.506 ... -41.456  69.592 -81.004]
  [-34.553  65.

In [78]:
predictions = model.predict(x_test)

label_predictions = []

for prediction in predictions:
    if prediction[0] > prediction[1]:
        label_predictions.append(0)
    else: label_predictions.append(1)
    
label_predictions = np.array(label_predictions)
label_test = np.array(label_test)
print(label_predictions)
print(np.array(label_test))

true_count = 0
for idx, label in enumerate(label_predictions):
    if label == label_test[idx]:
        true_count += 1
        
print("Accuracy on test set: ", true_count/(len(label_test)))

4/4 [==============================] - 0s 39ms/step
[1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0
 1 0 1 0 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 0 0 1 1 0 1
 0 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 1 0 1 0 1]
Accuracy on test set:  0.7117117117117117
